# Word Definitions Dataset

goal, to build a model that can reformat the definitions of words.` 
to be considered succsful this project will need to take a take a word from this data set and reformat the definition. ie, provide a desription that doesnt match the syntactical base that it started with. lexi (something) should match. The understanding of the word is not lost. but the slm will need to compare the 

In [8]:
import pandas as pd

#reference (1)
df = pd.read_csv("English_Dictionary.csv", nrows=4000)
# df = pd.read_csv("English_Dictionary.csv")

dataset = df
from IPython.core.display import display, HTML

# display(HTML(f"""<style>.dataframe td, .dataframe {{text-align: left;}}</style>{dataset.head(10).to_html()}"""))

df


C:\Users\grant\AppData\Local\Temp\ipykernel_13876\3545211112.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


,word,pos,definition
0,NaN,v,test
1,A,NaN,The first letter of the English and of many ot...
2,A,NaN,The name of the sixth tone in the model major ...
3,A,NaN,"An adjective, commonly called the indefinite a..."
4,A,NaN,"In each; to or for each; as, ""twenty leagues a..."
...,...,...,...
3995,Alcove,n.,"A recessed portion of a room, or a small room ..."
3996,Alcove,n.,"A small ornamental building with seats, or an ..."
3997,Alcove,n.,Any natural recess analogous to an alcove or r...
3998,Alcyon,n.,See Halcyon.


# Dataset Meaning/description

### shape:

- 188,539 rows and 3 columns
- size = 565,617 cells
- space = 14.4 MB

## Columns

- word: The word being defined
- part of speech: The type family the word is from
- definition: The definition of the word  



### Dataset description

the dataset contains every variation of a word, catagorised alphabetically within the csv file. 

- word:  
string based text depicting a globaly recognised word.
all starting with a capital and maintaining the correct grammer within multi word situations ("-")


The specific POS tags found in this dataset include:

prep.  

n.  

a.  

v. t.  

adv.  

p. p.  

v. i.  

n. pl.  

adv. & a.  

superl.  

v.  

v. t. & i.  

p. a.  

p. p. & a.  

a & n.  



- blank         :         Not specified

- n.            : Noun - a person, place, thing, or idea
- a.            : Adjective - a word that describes a noun  
- adv.          : Adverb - modifies a verb, adjective, or other adverb
- prep.         : Preposition - shows relationship between words
- v. t.         : Verb transitive - an action that requires a direct object
- n. pl.        : Noun, plural - plural form of a noun
- interj.       : Interjection - expresses emotion

- v / vb : Verb - an action or state of being  

- vbd : Verb, past tense  
- vbg : Verb, gerund or present participle  
- vbn : Verb, past participle  
- vbp : Verb, non-3rd person singular present  
- vbz : Verb, 3rd person singular present  
- nn : Noun, singular or mass  
- nns : Noun, plural  
- prp : Personal pronoun  
- in : preposition/subordinations  
- conj : conjunction
- adj : adjective
- adv : adverb
- pron : pronoun



### Standard Parts of Speech Reference

- Noun: A person, place, thing, or idea

- Pronoun: A word that replaces a noun
- Verb: An action or state of being
- Adjective: A word that describes a noun
- Adverb: A word that modifies a verb, adjective, or other adverb
- Preposition: Shows the relationship between a noun or pronoun and other words
- Conjunction: Connects words, phrases, or clauses
- Interjection: Expresses emotion

These categories help in understanding the function of words in communication and will be essential for building our reformatting model.


- Definition  
string text depicting the full definition of the rows word, punctuated and containing grammer cases (", - : . ") which will need to be addressed in processing. 


### check all POS tags

In [30]:
# check the pos definitions

pos_tags = dataset['pos'].unique()
for tag in pos_tags:
    print(tag)


prep.
n.
a.
v. t.
adv.
p. p.
v. i.
n. pl.
adv. & a.
superl.
v.
v. t. & i.
p. a.
p. p. & a.
a & n.
a. & adv.


# Assess the dataset for areas to clean

In [9]:
# Random data pulled from csv
import random
#reference (3)
filename = "English_Dictionary.csv"


n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_csv(filename, skiprows=skip)


display(HTML(f"""<style>.dataframe td, .dataframe {{text-align: left;}}</style>{df.head(50).to_html()}"""))


,word,pos,definition
0,Ab,n.,"The fifth month of the Jewish year according to the ecclesiastical reckoning, the eleventh by the civil computation, coinciding nearly with August."
1,Abactor,n.,One who steals and drives away cattle or beasts by herds or droves.
2,Abandum,n.,Anything forfeited or confiscated.
3,Abashedly,adv.,In an abashed manner.
4,Abate,v. t.,To reduce in estimation; to deprive.
5,Abatis,n.,Alt. of Abattis
6,Abdicate,v. i.,"To relinquish or renounce a throne, or other high office or dignity."
7,Abeyance,n.,Expectancy; condition of being undetermined.
8,Abeyancy,n.,Abeyance.
9,Abhominal,a.,Inhuman.


# from this, parameters to clean include:



- 1: remove empty definitions (check for NaN values in definition column)

- 2: remove 2 words definitions: by checking for definitions with only 1 space (whilst these may still fit criteria for testing, the training model is unlikely to benefit from them due to lack of context)

Abode,imp. & p. p.,of Abide
Abid,,of Abide
Abiding,p. pr. & vb. n.,of Abide
Abide,v. i.,To wait; to pause; to delay.

what to do with this, whilst a words definition is often directly linked to another word, structured via a past tense etc?????????????

- 3: check for multiple instances of a word with different definitions: checking for duplicate words in the "word" column (likely tie within step 1)

- 4: remove all punctuation within definitions

- 5: will be by using regex to replace all punctuation characters with an empty string

- 6: it appears words in () within definitions are secondary descriptive words, or are references to external events or concepts. these will need to be considered for removal

- 7: process all POS tags into a standard format, it seems some tags are using multiple tags. standardize these

- 8: shortern definitions to a max length, ie source which parts are unimportant and remove. some include examples in different contexts which may be unneeded

## notable

part of the data changes formatting midway

- Reenforce,v.,Something which reenforces or strengthens.
- Reenforce,v.,"That part of a cannon near the breech which is thicker than the rest of the piece, so as better to resist the force of the exploding powder. See Illust. of Cannon.Reenforce (v.)  (b) Reenforce (v.) An additional thickness of canvas, cloth, or the like, around an eyelet, buttonhole, etc.
- Reenforcement (n.) The act of reenforcing, or the state of being reenforced.
- Reenforcement (n.) That which reenforces; additional force; especially, additional troops or force to augment the strength of any army, or ships to strengthen a navy or fleet.
- Reengage (v. t. & i.) To engage a second time or again.

as shown the csv stops using commas to seperate columns and instead uses parentheses. this will need to be addressed in cleaning. the removeal or readaptation will be considered.


- some data contains stages of exlpination, consideration will need to be taken to determine if this is going to hinder or aid the model



# Stage 1:

### Check and clear empty definitions from dataset

- drop rows with no word
- drop rows with no pos
- drop rows with no deffinition

In [10]:
# check for empty definition, and empty words
print("---")
print("stage 1 status: words or definitions with missing data found below : ")
print(" ") 
a = 0 
b = 0
c = 0

removed_word = []
removed_def = []
removed_pos = []
for index, row in dataset.iterrows():
    if pd.isna(row['definition']):
        # print(row['word'] + " : empty definition found")
        removed_word.append(row['word'])
        a += 1


    if pd.isna(row["word"]):
        # print(row['definition'] + " : empty word found")
        removed_def.append(row['definition'])
        b += 1
    if pd.isna(row["pos"]):
        # print(row['word'] + " : has no pos tag")
        removed_pos.append(row['word'])
        c += 1

print("---")

# remove empty definitions and empty words

dataset = dataset.dropna(subset=['word', 'pos', 'definition'])



i = 0
j = 0
k = 0

for index, row in dataset.iterrows():
    if pd.isna(row['definition']):
        print(row['word'])
        print("empty definition found")
        i += 1

    if pd.isna(row["word"]):
        print(row['definition'])
        print("empty word found")
        j += 1

    if pd.isna(row["pos"]):
        print(row['word'] + " has no pos tag")
        k += 1

print(" ")
print("stage 1 cleaning complete: ")
print(" ")
print("words removed " + str(a) + " : " + str(removed_word))
print("definitions removed : " + str(b) + " : " + str(removed_def))
print("words with no pos tags removed : " + str(c) + " : " + str(removed_pos))
print(str(i) + " empty definitions found after cleaning")
print(str(j) + " empty words found after cleaning")
print(str(k) + " empty pos tags found after cleaning")
# # # confirms they are removed 

---
stage 1 status: words or definitions with missing data found below : 
 
---
 
stage 1 cleaning complete: 
 
words removed 0 : []
definitions removed : 1 : ['test']
words with no pos tags removed : 96 : ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A-', 'A 1', "Aaron's rod", "Aaron's rod", 'Ab-', 'Abb wool', 'A B C', 'A B C', 'A B C', 'Abegge', 'Abid', 'Abit', 'Ablative', '-able', 'Ablet', 'Ablen', '-ably', 'Abode', 'Abought', 'Absque hoc', 'A cappella', 'A cappella', 'Account book', 'Accoutred', 'Accoutring', 'Acorn cup', 'Ad-', "Adam's apple", 'Ad captandum', 'Adder fly/', "Addison's disease", 'Aden-', 'Adeno-', 'Ad hominem', 'Ad infinitum', 'Ad interim', 'Adjacence', 'Adjacency', 'Adjacency', 'Adjectitious', 'Ad libitum', 'Adoor', 'Adoors', 'Ad valorem', 'Advertence', 'Advertency', 'Ae', 'Ae', 'Aero-', 'Aethiops mineral', 'A fortiori', 'After damp', 'Agnus castus', 'Agnus Dei', 'Agnus Dei', 'Agnus Dei', 'Agronomic', 'Agronomical', "Aich's metal", "Ain't", 'Air bed', 'Air bladder', 'Air bl

### after assessing the results removal of the pos may not be the best approach values contain important words:
- adjecency, agronomic, A.. as this is just one letter its hard to predict the impact of removing these
- the dataset ran with nrows 4000 which is a fraction of the total 180k

- the cell below resets the csv file 

In [11]:
# Reload csv for actual cleaning 

df = pd.read_csv("English_Dictionary.csv", nrows=2000)
# df = pd.read_csv("English_Dictionary.csv")

dataset = df

# check for empty definition, and empty words
print("---")
print("stage 1 status: words or definitions with missing data found below : ")
print(" ") 
a = 0 
b = 0

removed_word = []
removed_def = []
for index, row in dataset.iterrows():
    if pd.isna(row['definition']):
        # print(row['word'] + " : empty definition found")
        removed_word.append(row['word'])
        a += 1


    if pd.isna(row["word"]):
        # print(row['definition'] + " : empty word found")
        removed_def.append(row['definition'])
        b += 1

print("---")

# remove empty definitions and empty words

dataset = dataset.dropna(subset=['word', 'pos', 'definition'])



i = 0
j = 0

for index, row in dataset.iterrows():
    if pd.isna(row['definition']):
        print(row['word'])
        print("empty definition found")
        i += 1

    if pd.isna(row["word"]):
        print(row['definition'])
        print("empty word found")
        j += 1


print(" ")
print("stage 1 cleaning complete: ")
print(" ")
print("words removed " + str(a) + " : " + str(removed_word))
print("definitions removed : " + str(b) + " : " + str(removed_def))
print(str(i) + " empty definitions found after cleaning")
print(str(j) + " empty words found after cleaning")
# # # confirms they are removed 

---
stage 1 status: words or definitions with missing data found below : 
 
---
 
stage 1 cleaning complete: 
 
words removed 0 : []
definitions removed : 1 : ['test']
0 empty definitions found after cleaning
0 empty words found after cleaning


# check for 2 word definitions which seem to be references to other words

In [12]:
# find words with 2 word definitions
small_defs = []
small_word = []
count_dup = []

for index, row in dataset.iterrows():
    if (row["definition"]).count(" ") == 1:
        # print(row['word'], " ", row['definition'])
        small_word.append(row['word'])
        small_defs.append(row['definition'])

for word in small_word:
    dup_count = (dataset["word"] == word).sum()
    
    count_dup.append((str(dup_count)) if dup_count > 1 else (0))
print(small_word)
print(small_defs)
print(count_dup)
print("---")
print(str(len(small_word)) + ": words that have 2 word definitions")


['Aback', 'Abaculi', 'Abacuses', 'Abaci', 'Abada', 'Aband', 'Abandoned', 'Abandoning', 'Abandon', 'Abandoned', 'Abanet', 'Abased', 'Abasing', 'Abased', 'Abasedly', 'Abashed', 'Abashing', 'Abated', 'Abating', 'Abate', 'Abattoirs', 'Abawed', 'Abbacies', 'Abbeys', 'Abbreviated', 'Abbreviating', 'Abbreviate', 'Abdicated', 'Abdicating', 'Abdominals', 'Abdominales', 'Abduced', 'Abducing', 'Abducted', 'Abducting', 'Abetted', 'Abetting', 'Abhorred', 'Abhorring', 'Abhorrently', 'Abode', 'Abiding', 'Abiding', 'Abilities', 'Abyme', 'Abjectly', 'Abjugate', 'Abjuratory', 'Abjured', 'Abjuring', 'Ablactate', 'Abluent', 'Ablush', 'Abnegated', 'Abnegating', 'Abnormalities', 'Abnormality', 'Abnormities', 'Abnormous', 'Aboard', 'Abode', 'Aboding', 'Abolished', 'Abolishing', 'Abominated', 'Abominating', 'Aborsement', 'Abortifacient', 'Abounded', 'Abounding', 'Abox', 'Abraded', 'Abrading', 'Abranchiate', 'Abrase', 'Abrasive', 'Abray', 'Abregge', 'Abrenounce', 'Abricock', 'Abridged', 'Abridging', 'Abrogate'

##### These three arrays show us which words are appearing once or more. the significance of this is that any words that appear more than 0 (once) are multiple definitions of the same word.
##### Any words that appear "0" are single entries of that word. This means that it is likely referring to another word in the dataset (as the data is collecting words that only have one space)


# Isolate and drop the single entries

In [13]:
# Drop words with 2 word definitions, that appear only once
single_words_entry = []


for count_dup in enumerate(count_dup):
    if count_dup[1] == 0:
        single_words_entry.append(small_word[count_dup[0]])

print(single_words_entry)

for word in single_words_entry:
    indices_to_drop = dataset[dataset['word'] == word].index
    dataset = dataset.drop(indices_to_drop)
    print("dropped: " + word)

['Abaculi', 'Abacuses', 'Abaci', 'Abada', 'Abandoning', 'Abanet', 'Abasing', 'Abasedly', 'Abashed', 'Abashing', 'Abated', 'Abating', 'Abattoirs', 'Abawed', 'Abbacies', 'Abbeys', 'Abbreviating', 'Abdicated', 'Abdicating', 'Abdominals', 'Abduced', 'Abducing', 'Abducted', 'Abducting', 'Abetted', 'Abetting', 'Abhorred', 'Abhorrently', 'Abilities', 'Abyme', 'Abjectly', 'Abjugate', 'Abjuratory', 'Abjured', 'Abjuring', 'Ablactate', 'Ablush', 'Abnegated', 'Abnegating', 'Abnormalities', 'Abnormities', 'Abnormous', 'Aboding', 'Abolished', 'Abolishing', 'Abominated', 'Abominating', 'Aborsement', 'Abounded', 'Abounding', 'Abox', 'Abraded', 'Abrading', 'Abranchiate', 'Abrase', 'Abrasive', 'Abray', 'Abregge', 'Abrenounce', 'Abricock', 'Abridged', 'Abridging', 'Abrogated', 'Abrogating', 'Abscesses', 'Abscision', 'Abscisses', 'Absciss', 'Abscissas', 'Abscissae', 'Absconded', 'Absconding', 'Absented', 'Absenting', 'Absis', 'Absolved', 'Absolving', 'Absorbed', 'Abstained', 'Abstaining', 'Abstinently', '

This has removed all 2 word definitions from the dataset, that have one occurrence. (words referring to other words as definitions)
The next step will be to remove the remaining short words from the dataset. The difference here is that these words have multiple definitions, so we need to keep the others that are present.

In [14]:
# Check unique words have been removed same process as before

small_defs = []
small_word = []
count_dup = []

for index, row in dataset.iterrows():
    if (row["definition"]).count(" ") == 1:
        # print(row['word'], " ", row['definition'])
        small_word.append(row['word'])
        small_defs.append(row['definition'])

for word in small_word:
    dup_count = (dataset["word"] == word).sum()
    
    count_dup.append((str(dup_count)) if dup_count > 1 else (0))
print(small_word)
print(small_defs)
print(count_dup)
print("---")
print(str(len(small_word)) + ": words that have 2 word definitions")


['Aback', 'Aband', 'Abandoned', 'Abandon', 'Abandoned', 'Abased', 'Abased', 'Abate', 'Abbreviated', 'Abbreviate', 'Abdominales', 'Abhorring', 'Abode', 'Abiding', 'Abiding', 'Abluent', 'Abnormality', 'Aboard', 'Abode', 'Abortifacient', 'Abrogate', 'Absolute', 'Absolvent', 'Absorbing', 'Abstersive', 'Abstersive', 'Abstract', 'Abstracted', 'Absurd', 'Abuse', 'Academician', 'Academist', 'Acantha', 'Acceptance', 'Acceptant', 'Acceptant', 'Accepter', 'Accipitres', 'Acclaim', 'Accommodating', 'Accomplice', 'Accomplished', 'According', 'According', 'Accosted', 'Accost', 'Accountant', 'Accresce', 'Accrescent', 'Accrete', 'Accumulate', 'Accused', 'Accustomed', 'Accustom', 'Acerate', 'Achate', 'Achate', 'Aching', 'Acinus', 'Acknow', 'Acorn', 'Acquainted', 'Acquittance', 'Acraze', 'Acting', 'Acuate', 'Aculeus', 'Adamant', 'Adapt']
['An abacus.', 'To abandon.', 'of Abandon', 'Abandonment; relinquishment.', 'Forsaken, deserted.', 'of Abase', 'Lowered; humbled.', 'To blunt.', 'of Abbreviate', 'An abr

we have removed all unique words that have two word definitions as shows bellow the 3rd array the value has significantly dropped

next remove all words with short definitions, only removing the items with the two word as these ones contain multiple definitions.


In [15]:
# drop all words with 2 word definitions
iso_words_entry = []

for row in dataset.iterrows():
    if row[1]["word"] in small_word and (row[1]["definition"]).count(" ") == 1:
        dataset = dataset.drop(row[0])
        print("dropped: " + row[1]["word"])





dropped: Aback
dropped: Aband
dropped: Abandoned
dropped: Abandon
dropped: Abandoned
dropped: Abased
dropped: Abased
dropped: Abate
dropped: Abbreviated
dropped: Abbreviate
dropped: Abdominales
dropped: Abhorring
dropped: Abode
dropped: Abiding
dropped: Abiding
dropped: Abluent
dropped: Abnormality
dropped: Aboard
dropped: Abode
dropped: Abortifacient
dropped: Abrogate
dropped: Absolute
dropped: Absolvent
dropped: Absorbing
dropped: Abstersive
dropped: Abstersive
dropped: Abstract
dropped: Abstracted
dropped: Absurd
dropped: Abuse
dropped: Academician
dropped: Academist
dropped: Acantha
dropped: Acceptance
dropped: Acceptant
dropped: Acceptant
dropped: Accepter
dropped: Accipitres
dropped: Acclaim
dropped: Accommodating
dropped: Accomplice
dropped: Accomplished
dropped: According
dropped: According
dropped: Accosted
dropped: Accost
dropped: Accountant
dropped: Accresce
dropped: Accrescent
dropped: Accrete
dropped: Accumulate
dropped: Accused
dropped: Accustomed
dropped: Accustom
droppe

In [16]:
# final test to see the current 2 word situation

small_defs = []
small_word = []
count_dup = []

for index, row in dataset.iterrows():
    if (row["definition"]).count(" ") == 1:
        # print(row['word'], " ", row['definition'])
        small_word.append(row['word'])
        small_defs.append(row['definition'])

for word in small_word:
    dup_count = (dataset["word"] == word).sum()
    
    count_dup.append((str(dup_count)) if dup_count > 1 else (0))
print("words with 2 word def = " + str(small_word))
print("definitions with 2 word def = " + str(small_defs))
print("count of definitions with 2 word def = " + str(count_dup))
print("---")
print(str(len(small_word)) + ": words that have 2 word definitions")


words with 2 word def = []
definitions with 2 word def = []
count of definitions with 2 word def = []
---
0: words that have 2 word definitions


## Clean the grammar found within definitions

In [27]:
# Remove grammar from the definitions.

import re
import string


# reference(3)



def clean_definition(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

dataset['definition'] = dataset['definition'].apply(clean_definition)

# Display the cleaned dataset
display(HTML(f"""<style>.dataframe td, .dataframe {{text-align: left;}}</style>{dataset.head(10).to_html()}"""))




,word,pos,definition
5,A,prep.,in on at by
6,A,prep.,in process of in the act of into to used with verbal substantives in ing which begin with a consonant this is a shortened form of the preposition an which was used before the vowel sound as in a hunting a building a begging
12,Aam,n.,a dutch and german measure of liquids varying in different cities being at amsterdam about wine gallons at antwerp at hamburg
13,Aard-vark,n.,an edentate mammal of the genus orycteropus somewhat resembling a pig common in some parts of southern africa it burrows in the ground and feeds entirely on ants which it catches with its long slimy tongue
14,Aard-wolf,n.,a carnivorous quadruped proteles lalandii of south africa resembling the fox and hyena see proteles
15,Aaronic,a.,alt of aaronical
16,Aaronical,a.,pertaining to aaron the first high priest of the jews
20,Ab,n.,the fifth month of the jewish year according to the ecclesiastical reckoning the eleventh by the civil computation coinciding nearly with august
21,Abaca,n.,the manilahemp plant musa textilis also its fiber see manila hemp under manila
22,Abacinate,v. t.,to blind by a redhot metal plate held before the eyes


SyntaxError: invalid syntax (4267813958.py, line 1)

# stage 1 cleaning complete


### next steps will be highlight any other issues with the dataset


frst i will plug in a random chunk (from reference) to just visually see if they're are any bizzare instances